In [1]:
#conda install -c conda-forge geopy

In [1]:
# Importing required module 
from geopy.geocoders import Nominatim 
import pandas as pd

In [2]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [155]:
#Import all the postal codes of Tokyo
zip_all = pd.read_excel("tokyo_postal_code.xlsx", header = None, names = ["zip_code", "都", "区", "町", "Prefecture", "District", "Area"])
zip_all.shape

(4005, 7)

In [156]:
zip_all.head()

,zip_code,都,区,町,Prefecture,District,Area
0,1000000,東京都,千代田区,以下に掲載がない場合,TOKYO TO,CHIYODA KU,IKANIKEISAIGANAIBAAI
1,1020072,東京都,千代田区,飯田橋,TOKYO TO,CHIYODA KU,IIDABASHI
2,1020082,東京都,千代田区,一番町,TOKYO TO,CHIYODA KU,ICHIBANCHO
3,1010032,東京都,千代田区,岩本町,TOKYO TO,CHIYODA KU,IWAMOTOCHO
4,1010047,東京都,千代田区,内神田,TOKYO TO,CHIYODA KU,UCHIKANDA


In [157]:
# append geographical coordinate
zip_all["lat"] = None
zip_all["long"] = None
zip_all.head()

,zip_code,都,区,町,Prefecture,District,Area,lat,long
0,1000000,東京都,千代田区,以下に掲載がない場合,TOKYO TO,CHIYODA KU,IKANIKEISAIGANAIBAAI,None,None
1,1020072,東京都,千代田区,飯田橋,TOKYO TO,CHIYODA KU,IIDABASHI,None,None
2,1020082,東京都,千代田区,一番町,TOKYO TO,CHIYODA KU,ICHIBANCHO,None,None
3,1010032,東京都,千代田区,岩本町,TOKYO TO,CHIYODA KU,IWAMOTOCHO,None,None
4,1010047,東京都,千代田区,内神田,TOKYO TO,CHIYODA KU,UCHIKANDA,None,None


In [158]:
# fetch geographical coordinate
for i, code, district, area in zip(range(zip_all.shape[0]), zip_all["zip_code"], zip_all["District"], zip_all["Area"]):
    print("converting row {}".format(i))
    
    #use new agent everytime to evade timeout
    geolocator = Nominatim(user_agent="tokyo-coder" + str(i))
    
    # save the data in the dataframe
    location = geolocator.geocode("{}, {}, Tokyo".format(code, area, district))
    if location:
        zip_all.loc[i, "lat"] = location.latitude
        zip_all.loc[i, "long"] = location.longitude


converting row 0
converting row 1
converting row 2
converting row 3
converting row 4
converting row 5
converting row 6
converting row 7
converting row 8
converting row 9
converting row 10
converting row 11
converting row 12
converting row 13
converting row 14
converting row 15
converting row 16
converting row 17
converting row 18
converting row 19
converting row 20
converting row 21
converting row 22
converting row 23
converting row 24
converting row 25
converting row 26
converting row 27
converting row 28
converting row 29
converting row 30
converting row 31
converting row 32
converting row 33
converting row 34
converting row 35
converting row 36
converting row 37
converting row 38
converting row 39
converting row 40
converting row 41
converting row 42
converting row 43
converting row 44
converting row 45
converting row 46
converting row 47
converting row 48
converting row 49
converting row 50
converting row 51
converting row 52
converting row 53
converting row 54
converting row 55
co

In [9]:
#save the current data
#zip_all.to_csv("tokyo_zip_latlong.csv")

In [10]:
#zip_all = pd.read_csv("tokyo_zip_latlong.csv", index_col = 0)
#zip_all.shape

In [18]:
# remove rows which do not have lat and long info
zip_filtered = zip_all.dropna()
print("{} geographical coordinates are obtained successfully.".format(zip_filtered.shape[0]))

1035 geographical coordinates are obtained successfully


Fetch venues info by Folium

In [13]:
#Folium credentials

CLIENT_ID = 'XF1O1VAJQ1PVM0JZDGZP5DLGC0STTBPDGLDWVKDADMWWLKC4' # your Foursquare ID
CLIENT_SECRET = 'BUI4S2QZWXPLJCZC5BJON0UD2TIIXN30BVR43FL3JGSR40RD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XF1O1VAJQ1PVM0JZDGZP5DLGC0STTBPDGLDWVKDADMWWLKC4
CLIENT_SECRET:BUI4S2QZWXPLJCZC5BJON0UD2TIIXN30BVR43FL3JGSR40RD


In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    # inside the function, venues in all neighborhoods in our data are obtained iteratively.
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        
        except KeyError:
            venues_list.append([(
                name, 
                lat, 
                lng,
                None,
                None,
                None,
                None)])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [24]:
Tokyo_venues = getNearbyVenues(names = zip_filtered["Area"],
                               latitudes = zip_filtered["lat"],
                               longitudes = zip_filtered["long"])

ICHIBANCHO
IWAMOTOCHO
UCHIKANDA
UCHISAIWAICHO
KAJICHO
KANDASURUGADAI
KIOICHO
KUDANKITA
SOTOKANDA
CHIYODA
HIGASHIKANDA
HITOTSUBASHI(1-CHOME)
HITOTSUBASHI(2-CHOME)
HIRAKAWACHO
FUJIMI
YURAKUCHO
ROKUBANCHO
AKASHICHO
IRIFUNE
KYOBASHI
GINZA
SHINKAWA
SHINTOMI
TSUKISHIMA
TSUKIJI
TSUKUDA
TOYOMICHO
NIHOMBASHIKOAMICHO
HATCHOBORI
MINATO
YAESU(2-CHOME)
AZABUDAI
KAIGAN(1.2-CHOME)
KAIGAN(3-CHOME)
KITAAOYAMA
SHIBA(1-3-CHOME)
SHIBAURA(1-CHOME)
SHIBAKOEN
SHIBADAIMON
SHIROKANE
SHIROKANEDAI
SHIMBASHI
DAIBA
TAKANAWA
NISHIAZABU
NISHISHIMBASHI
MINAMIAOYAMA
MOTOAZABU
ARAKICHO
OKUBO
KAGURAZAKA
KATAMACHI
KABUKICHO
KITAMACHI
SAMONCHO
SHINANOMACHI
SUMIYOSHICHO
DAIKYOCHO
TAKADANOBABA
TSUKUDOHACHIMANCHO
TENJINCHO
TOMIHISACHO
NAKAI
NAKACHO
NISHIOCHIAI
HYAKUNINCHO
BENTENCHO
MINAMICHO
YOCHOMACHI
YOTSUYA
WAKABA
WAKAMATSUCHO
WASEDA MINAMICHO
OTSUKA
KASUGA
KOISHIKAWA
KORAKU
KOHINATA
SUIDO
SEKIGUCHI
SENGOKU
SENDAGI
NISHIKATA
NEZU
HAKUSAN(1-CHOME)
HAKUSAN(2-5-CHOME)
HONKOMAGOME
HONGO
MUKOGAOKA
MEJIRODAI
YAYOI
YUSHIMA
AKIHA

In [27]:
#save the venues data in a csv file
#Tokyo_venues.to_csv("tokyo_venues.csv", index = False)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ICHIBANCHO,35.729056,139.378416,None,None,None,None
1,IWAMOTOCHO,35.695600,139.775379,None,None,None,None
2,UCHIKANDA,35.691038,139.767290,None,None,None,None
3,UCHISAIWAICHO,35.669426,139.755460,None,None,None,None
4,KAJICHO,35.691689,139.771942,None,None,None,None
